In this tutorial, we'll demonstrate how to use OpenAI functioncalling.  We'll use the GPT-3.5 model to identify and call a function in the context of a conversation. We will use a simple function that returns a string as a response.

Let's import the required libraries

In [18]:
# Load environment variables from a .env file
from dotenv import load_dotenv
load_dotenv()

# Importing the necessary library for OpenAI API
import openai
import os
import json

# Define your OpenAI API key 
client = openai.OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key = os.environ.get('API_KEY')
)

#### Step 1: Define the Function

First, let's define a simple Python function that takes a string as an argument and returns a string as a response.

In [22]:
def restaurant_localizer(restaurant, location):
    return f"Function called with restaurant {restaurant} from location {location}"

#### Step 2: Define the Tools

Next, we define the `tools` list. This list contains the functions that the assistant can use. Each function is described by its name, description, and parameters.

In [23]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "restaurant_localizer",
            "description": "A simple function that localizes a restaurant",
            "parameters": {
                "type": "object",
                "properties": {
                    "restaurant": {
                        "type": "string",
                        "description": "The restaurant name"
                    },
                    "location": {
                        "type": "string",
                        "description": "The location of the restaurant"
                    }
                },
                "required": ["restaurant", "location"]
            }
        }
    }
]

#### Step 3: Define the Initial Conversation Messages

We define the initial conversation messages. The first message is from the system, describing the role of the assistant. The second message is from the user, requesting the function call. Feel free to change the argument!

In [28]:
messages = [
    {"role": "system", "content": "You are an assistant that localizes restaurants."},
    {"role": "user", "content": "Localize the restaurant 'Pizza Place' in 'San Francisco'."}
]

#### Step 4: Generate a Response from the Model

We call `openai.ChatCompletion.create()` (for openai < 1.0) or `openai.chat.completions.create()` (for openai > 1.0) to generate a response from the GPT-3.5 model. The model, messages, and tools are passed as arguments. The `tool_choice` is set to "auto", allowing the model to choose which tool (function) to use.

In [29]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

#### Step 5: Extract the Tool Calls and Call the Function

We extract any tool calls from the response. 

- If there are any tool calls, we map the tool names to their corresponding Python functions. 
- For each tool call in the response, we extract the function name and call the corresponding Python function with the arguments provided by the model.

In [31]:
tool_calls = response.choices[0].message.tool_calls

if tool_calls:
    available_functions = {
        "restaurant_localizer": restaurant_localizer,
    }

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)

        if function_name == 'restaurant_localizer':
            function_response = restaurant_localizer(
                restaurant=function_args.get("restaurant"),
                location=function_args.get("location"),
            )

        messages.append({
            "role": "assistant",
            "content": function_response,
        })

#### Step 6: Print the Conversation

Finally, we print the conversation messages to see the function call and response in the context of the conversation.

In [32]:
for message in messages:
    print(f"{message['role']}: {message['content']}")

system: You are an assistant that localizes restaurants.
user: Localize the restaurant 'Pizza Place' in 'San Francisco'.
assistant: Function called with restaurant Pizza Place from location San Francisco
assistant: Function called with restaurant Pizza Place from location San Francisco
